In [ ]:
!pip install av
!wget -c https://du7u4d2q1sjz6.cloudfront.net/datas/dog.zip
!unzip dog.zip

In [1]:
import av


class WebmWriter:
    def __init__(self, path, width, height, fps=25):
        self.container = av.open(path, mode="w")
        self.stream = self.container.add_stream("vp9", rate=fps)
        self.stream.pix_fmt = "yuva420p"
        self.stream.width = width
        self.stream.height = height

    def write(self, img):
        frame = av.VideoFrame.from_ndarray(img, format="rgba")
        for packet in self.stream.encode(frame):
            self.container.mux(packet)

    def __del__(self):
        self.close()

    def close(self):
        # Flush stream
        for packet in self.stream.encode():
            self.container.mux(packet)

        # Close the file
        self.container.close()





In [2]:
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import Video

frames = sorted(glob.glob("./dog/*.jpg"))
frames_alpha = sorted(glob.glob("./dog/*.png"))

writer = WebmWriter("./test.webm", 854, 480)

for image, alpha in tqdm(zip(frames, frames_alpha)):
    image = np.array(Image.open(image).convert("RGB"))
    alpha = np.array(Image.open(alpha)) * 255
    out = np.concatenate([image, alpha[:, :, np.newaxis]], 2)
    writer.write(out)

writer.close()

display(Video("./test.webm"))


60it [00:10,  5.47it/s]
